<a href="https://colab.research.google.com/github/yeungjosh/resnet-perturbations-riselab/blob/master/pytorch_resnet_mnist_frank_wolfe.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ResNet for MNIST in PyTorch

In [1]:
!pip install --upgrade --force-reinstall --quiet git+https://github.com/ZIB-IOL/StochasticFrankWolfe.git@arXiv-2010.07243v2
!pip install --quiet barbar

In [2]:
from torchvision.models.resnet import ResNet, BasicBlock
from torchvision.datasets import MNIST
from tqdm.autonotebook import tqdm
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score
import inspect
import time

import torch

from torch import nn, optim
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.autograd import Variable
from torchvision.transforms import Compose, ToTensor, Normalize, Resize
from torch.utils.data import DataLoader
from copy import copy, deepcopy
import numpy as np

import frankwolfe.pytorch as fw


In [3]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [4]:
def save_last_model(input_model):
  model_save_name = 'resnet_frank_wolfe_last.pkl'
  path = F"/content/gdrive/My Drive/{model_save_name}" 
  torch.save(input_model, path)

In [5]:
class MnistResNet(ResNet):
    def __init__(self):
        super(MnistResNet, self).__init__(BasicBlock, [2, 2, 2, 2], num_classes=10)
        self.conv1 = torch.nn.Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
        
    def forward(self, x):
        return torch.softmax(super(MnistResNet, self).forward(x), dim=-1)


In [15]:

def getData(name='cifar10', train_bs=128, test_bs=1000):    
    if name == 'svhn':
        train_loader = torch.utils.data.DataLoader(
    datasets.SVHN('../data', split='extra', download=True,
                   transform=transforms.Compose([
                       transforms.ToTensor()
                   ])),
    batch_size=train_bs, shuffle=True)
        test_loader = torch.utils.data.DataLoader(
    datasets.SVHN('../data', split='test', download=True,transform=transforms.Compose([
                       transforms.ToTensor()
                   ])),
    batch_size=test_bs, shuffle=False)
    if name == 'mnist':
        train_loader = torch.utils.data.DataLoader(
            datasets.MNIST('../data', train=True, download=True,
                           transform=transforms.Compose([
                               transforms.ToTensor(),
                               transforms.Normalize((0.1307,), (0.3081,))
                           ])),
            batch_size=train_bs, shuffle=True)
        test_loader = torch.utils.data.DataLoader(
            datasets.MNIST('../data', train=False, transform=transforms.Compose([
                               transforms.ToTensor(),
                               transforms.Normalize((0.1307,), (0.3081,))
                           ])),
            batch_size=test_bs, shuffle=False)
    if name == 'emnist':
        train_loader = torch.utils.data.DataLoader(
            datasets.EMNIST('../data', train=True, download=True, split='balanced',
                           transform=transforms.Compose([
                               transforms.ToTensor(),
                               transforms.Normalize((0.1751,), (0.3267,))
                           ])),
            batch_size=train_bs, shuffle=True)
        test_loader = torch.utils.data.DataLoader(
            datasets.EMNIST('../data', train=False, split='balanced', transform=transforms.Compose([
                               transforms.ToTensor(),
                               transforms.Normalize((0.1751,), (0.3267,))
                           ])),
            batch_size=test_bs, shuffle=False)
    if name == 'cifar10':
        transform_train = transforms.Compose([
        transforms.RandomCrop(32, padding=4),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
    ])
        transform_test = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
    ])
        trainset = datasets.CIFAR10(root='../data', train=True, download=True, transform=transform_train)
        train_loader = torch.utils.data.DataLoader(trainset, batch_size=train_bs, shuffle=True)
        testset = datasets.CIFAR10(root='../data', train=False, download=False, transform=transform_test)
        test_loader = torch.utils.data.DataLoader(testset, batch_size=test_bs, shuffle=False)
    if name == 'cifar100':
        transform_train = transforms.Compose([
        transforms.RandomCrop(32, padding=4),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
    ])
        transform_test = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
    ])
        trainset = datasets.CIFAR100(root='../data', train=True, download=True, transform=transform_train)
        train_loader = torch.utils.data.DataLoader(trainset, batch_size=train_bs, shuffle=True)
        testset = datasets.CIFAR100(root='../data', train=False, download=False, transform=transform_test)
        test_loader = torch.utils.data.DataLoader(testset, batch_size=test_bs, shuffle=False)
    if name == 'tinyimagenet':      
        normalize = transforms.Normalize(mean=[0.44785526394844055, 0.41693055629730225, 0.36942949891090393],
                                     std=[0.2928885519504547, 0.28230994939804077, 0.2889912724494934])
        train_dataset = datasets.ImageFolder(
        '../data/tiny-imagenet-200/train',
        transforms.Compose([
            transforms.RandomCrop(64, padding=4),
            transforms.RandomHorizontalFlip(),
            transforms.ToTensor(),
            normalize,
        ]))
        train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=train_bs, shuffle=True, num_workers=4, pin_memory=False)
        test_dataset = datasets.ImageFolder(
        '../data/tiny-imagenet-200/val',
        transforms.Compose([
            transforms.ToTensor(),
            normalize,
        ]))
        test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=test_bs, shuffle=False)
    return train_loader, test_loader

In [16]:
def calculate_metric(metric_fn, true_y, pred_y):
    if "average" in inspect.getfullargspec(metric_fn).args:
        return metric_fn(true_y, pred_y, average="macro")
    else:
        return metric_fn(true_y, pred_y)
    
def print_scores(p, r, f1, a, batch_size):
    for name, scores in zip(("precision", "recall", "F1", "accuracy"), (p, r, f1, a)):
        print(f"\t{name.rjust(14, ' ')}: {sum(scores)/batch_size:.4f}")

In [17]:
class RetractionLR(torch.optim.lr_scheduler._LRScheduler):
    """
    Retracts the learning rate as follows. Two running averages are kept, one of length n_close, one of n_far. Adjust
    the learning_rate depending on the relation of far_average and close_average. Decrease by 1-retraction_factor.
    Increase by 1/(1 - retraction_factor*growth_factor)
    """
    def __init__(self, optimizer, retraction_factor=0.3, n_close=5, n_far=10, lowerBound=1e-5, upperBound=1, growth_factor=0.2, last_epoch=-1):
        self.retraction_factor = retraction_factor
        self.n_close = n_close
        self.n_far = n_far
        self.lowerBound = lowerBound
        self.upperBound = upperBound
        self.growth_factor = growth_factor

        assert (0 <= self.retraction_factor < 1), "Retraction factor must be in [0, 1[."
        assert (0 <= self.lowerBound < self.upperBound <= 1), "Bounds must be in [0, 1]"
        assert (0 < self.growth_factor <= 1), "Growth factor must be in ]0, 1]"

        self.closeAverage = RunningAverage(self.n_close)
        self.farAverage = RunningAverage(self.n_far)

        super(RetractionLR, self).__init__(optimizer, last_epoch)

    def update_averages(self, loss):
        self.closeAverage(loss)
        self.farAverage(loss)

    def get_lr(self):
        if not self._get_lr_called_within_step:
            warnings.warn("To get the last learning rate computed by the scheduler, "
                          "please use `get_last_lr()`.", UserWarning)

        factor = 1
        if self.farAverage.is_complete() and self.closeAverage.is_complete():
            if self.closeAverage.result() > self.farAverage.result():
                # Decrease the learning rate
                factor = 1 - self.retraction_factor
            elif self.farAverage.result() > self.closeAverage.result():
                # Increase the learning rate
                factor = 1./(1 - self.retraction_factor*self.growth_factor)

        return [max(self.lowerBound, min(factor * group['lr'], self.upperBound)) for group in self.optimizer.param_groups]

class RunningAverage(object):
    """Tracks the running average of n numbers"""
    def __init__(self, n):
        self.n = n
        self.reset()

    def reset(self):
        self.sum = 0
        self.avg = 0
        self.entries = []

    def result(self):
        return self.avg

    def get_count(self):
        return len(self.entries)

    def is_complete(self):
        return len(self.entries) == self.n

    def __call__(self, val):
        if len(self.entries) == self.n:
            l = self.entries.pop(0)
            self.sum -= l
        self.entries.append(val)
        self.sum += val
        self.avg = self.sum / len(self.entries)

    def __str__(self):
        return str(self.avg)

class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self):
        self.reset()

    def reset(self):
        self.sum = 0
        self.count = 0
        self.avg = 0

    def result(self):
        return self.avg

    def __call__(self, val, n=1):
        """val is an average over n samples. To compute the overall average, add val*n to sum and increase count by n"""
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

    def __str__(self):
        return str(self.avg)
        

In [18]:
# Init model
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = MnistResNet().to(device)

In [19]:
#@title Choosing Lp-Norm constraints
#@markdown The following cell allows you to set Lp-norm constraints for the chosen network. For exact parameters both for the constraints and the optimizer see the last cell of this notebook.
ord =  "2" #@param [1, 2, 5, 'inf']
ord = float(ord)
value = 10 #@param {type:"number"}
mode = 'initialization' #@param ['initialization', 'radius', 'diameter']

assert value > 0

# Select constraints
constraints = fw.constraints.create_lp_constraints(model, ord=ord, value=value, mode=mode)

In [20]:
#@title Configuring the Frank-Wolfe Algorithm
#@markdown Choose momentum and learning rate rescaling, see Section 3.1 of [arXiv:2010.07243](https://arxiv.org/pdf/2010.07243.pdf).
momentum = 0.9 #@param {type:"number"}
rescale = 'gradient' #@param ['gradient', 'diameter', 'None']
rescale = None if rescale == 'None' else rescale

#@markdown Choose a learning rate for SFW. You can activate the learning rate scheduler which automatically multiplies the current learning rate by `lr_decrease_factor` every `lr_step_size epochs`
learning_rate = 0.1 #@param {type:"number"}
lr_scheduler_active = True #@param {type:"boolean"}
lr_decrease_factor = 0.1 #@param {type:"number"}
lr_step_size = 30 #@param {type:"integer"}

#@markdown You can also enable retraction of the learning rate, i.e., if enabled the learning rate is increased and decreased automatically depending on the two moving averages of different length of the train loss over the epochs.
retraction = True #@param {type:"boolean"}

assert learning_rate > 0
assert 0 <= momentum <= 1
assert lr_decrease_factor > 0
assert lr_step_size > 0


# Select optimizer
optimizer = fw.optimizers.SFW(params=model.parameters(), learning_rate=learning_rate, momentum=momentum, rescale=rescale)

In [23]:
from six.moves import urllib    
opener = urllib.request.build_opener()
opener.addheaders = [('User-agent', 'Mozilla/5.0')]
urllib.request.install_opener(opener)

In [24]:
start_ts = time.time()


epochs = 70

# train_loader, val_loader = get_data_loaders(256, 256)
train_loader, val_loader = getData(name='mnist', train_bs=128, test_bs=1000)


losses = []
loss_function = nn.CrossEntropyLoss()

# f_w
# initialize some necessary metrics objects
train_loss, train_accuracy = AverageMeter(), AverageMeter()
test_loss, test_accuracy = AverageMeter(), AverageMeter()

if lr_scheduler_active:
    scheduler = torch.optim.lr_scheduler.StepLR(optimizer=optimizer, step_size=lr_step_size, gamma=lr_decrease_factor)

if retraction:
    retractionScheduler = RetractionLR(optimizer=optimizer)

# function to reset metrics
def reset_metrics():
    train_loss.reset()
    train_accuracy.reset()

    test_loss.reset()
    test_accuracy.reset()

batches = len(train_loader)
val_batches = len(val_loader)
# keep best model
accuracies=[]
best_accuracy = 0
best_model = deepcopy(model)

# training loop + eval loop
for epoch in range(epochs):
    total_loss = 0
    progress = tqdm(enumerate(train_loader), desc="Loss: ", total=batches)
    model.train()
    # # lr decay
    # optimizer = exp_lr_scheduler(epoch, optimizer, decay_eff=0.1, decayEpoch=[15])

    for i, data in progress:
        X, y = data[0].to(device), data[1].to(device)
        
        model.zero_grad()
        outputs = model(X)
        loss = loss_function(outputs, y)

        loss.backward()
        optimizer.step(constraints=constraints)
        current_loss = loss.item()
        total_loss += current_loss
        progress.set_description("Loss: {:.4f}".format(total_loss/(i+1)))

        # train_loss(loss.item(), len(y))
        # train_accuracy(Utilities.categorical_accuracy(y_true=y_target, output=output), len(y_target))

    if lr_scheduler_active:
        scheduler.step()
    # if retraction:
    #     # Learning rate retraction
    #     retractionScheduler.update_averages(train_loss.result())
    #     retractionScheduler.step()    
    torch.cuda.empty_cache()
    
    val_losses = 0
    precision, recall, f1, accuracy = [], [], [], []
    
    model.eval()
    with torch.no_grad():
        for i, data in enumerate(val_loader):
            X, y = data[0].to(device), data[1].to(device)
            outputs = model(X)
            val_losses += loss_function(outputs, y)

            predicted_classes = torch.max(outputs, 1)[1]
            
            for acc, metric in zip((precision, recall, f1, accuracy), 
                                   (precision_score, recall_score, f1_score, accuracy_score)):
                acc.append(
                    calculate_metric(metric, y.cpu(), predicted_classes.cpu())
                )
    
    current_model_accuracy = sum(accuracy)/val_batches
    accuracies.append(current_model_accuracy)
    if current_model_accuracy > best_accuracy:
        best_model = deepcopy(model)
        best_accuracy=current_model_accuracy
        
    print(f"Epoch {epoch+1}/{epochs}, training loss: {total_loss/batches}, validation loss: {val_losses/val_batches}")
    print_scores(precision, recall, f1, accuracy, val_batches)
    losses.append(total_loss/batches)
    print('current_model_accuracy: ',current_model_accuracy)
    print('best_accuracy: ',best_accuracy)

save_last_model(model)
model_save_name = 'resnet_mnist_frank_wolfe_L2.pkl'
path = F"/content/gdrive/My Drive/{model_save_name}" 
torch.save(best_model, path)

print(losses)
print(f"Training time: {time.time()-start_ts}s")

Extracting ../data/MNIST/raw/train-images-idx3-ubyte.gz to ../data/MNIST/raw


Extracting ../data/MNIST/raw/train-labels-idx1-ubyte.gz to ../data/MNIST/raw




Extracting ../data/MNIST/raw/t10k-images-idx3-ubyte.gz to ../data/MNIST/raw


Extracting ../data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ../data/MNIST/raw
Processing...
Done!



/usr/local/lib/python3.7/dist-packages/torchvision/datasets/mnist.py:480: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:141.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


...


Epoch 66/70, training loss: 1.4617710001687252, validation loss: 1.4660080671310425
	     precision: 0.9952
	        recall: 0.9951
	            F1: 0.9951
	      accuracy: 0.9951
current_model_accuracy:  0.9951000000000001
best_accuracy:  0.9952



Epoch 67/70, training loss: 1.461779373795239, validation loss: 1.466129183769226
	     precision: 0.9951
	        recall: 0.9950
	            F1: 0.9950
	      accuracy: 0.9950
current_model_accuracy:  0.9950000000000001
best_accuracy:  0.9952



Epoch 68/70, training loss: 1.4617833455742548, validation loss: 1.466232180595398
	     precision: 0.9951
	        recall: 0.9950
	            F1: 0.9950
	      accuracy: 0.9950
current_model_accuracy:  0.9950000000000001
best_accuracy:  0.9952



Epoch 69/70, training loss: 1.4617799911926042, validation loss: 1.4661777019500732
	     precision: 0.9951
	        recall: 0.9950
	            F1: 0.9951
	      accuracy: 0.9950
current_model_accuracy:  0.9949999999999999
best_accuracy:  0.9952



Epoch 70/70, training loss: 1.4618027377992804, validation loss: 1.4661661386489868
	     precision: 0.9951
	        recall: 0.9950
	            F1: 0.9951
	      accuracy: 0.9950
current_model_accuracy:  0.9949999999999999
best_accuracy:  0.9952
[1.510880936691756, 1.4806050934008699, 1.477046595707631, 1.4736327855571756, 1.4720042898202501, 1.4715293981373183, 1.4713689790351558, 1.4698607265822161, 1.468794769569755, 1.4681079758764075, 1.4685766666428621, 1.4673032420022147, 1.4670141259236122, 1.4671001828301435, 1.4678839760294347, 1.467451391443769, 1.4675382914573654, 1.4675519659575114, 1.4666428037289618, 1.4661514380339111, 1.4676608985929347, 1.466084392594376, 1.4661129481756865, 1.466041695842865, 1.4667344825354212, 1.46620781335241, 1.4662178531130239, 1.4654675292562065, 1.467111259889501, 1.466518433363453, 1.4643576841618715, 1.4630393399866914, 1.4626762358618697, 1.462537113029057, 1.462368478144664, 1.4623686934330824, 1.4622848692224986, 1.4622223623780046, 1.4

In [ ]:
print('best_accuracy: ',best_accuracy)